In [1]:
import neo4jvis
import nxneo4j as nx
import random
import neomodel
from neo4j import GraphDatabase
from neo4jvis import *
from neo4jvis.model import *
from collections import defaultdict


In [2]:
DATABASE_URL = 'localhost:7687'
DABATASE_USER = 'neo4j'
DABATASE_PASS = 'pass'

In [3]:
from neomodel import (config, StructuredNode, StringProperty, IntegerProperty,
    UniqueIdProperty, RelationshipTo, StructuredRel)
from neomodel.relationship_manager import RelationshipManager

config.DATABASE_URL = f"bolt://{DABATASE_USER}:{DABATASE_PASS}@{DATABASE_URL}"

# Create graph

In [4]:
driver = GraphDatabase.driver(uri=f"bolt://{DATABASE_URL}",
                              auth=(DABATASE_USER, DABATASE_PASS))
                              #OR "bolt://localhost:7673"
                              #OR the cloud url

In [12]:
graph = StyledGraph(driver)
graph.generate_whole_graph()
out = graph.draw()
out

In [10]:
G = nx.Graph(driver)

In [11]:
G.delete_all()

In [8]:
LABEL_USUARIOS = "Usuarios"
LABEL_HASHTAGS = "Hashtags"
LABEL_TWITS = "Twits"
R_POSTS = "POSTS"
R_TAGS = "TAGS"

G.node_label = LABEL_USUARIOS
for i in range(3):
    G.add_node("usuario" + str(i))
G.node_label = LABEL_TWITS
for i in range(14):
    G.add_node("twit" + str(i))
G.node_label = LABEL_HASHTAGS
for i in range(8):
    G.add_node("hashtag" + str(i))
    
G.node_label = "Usuarios"
for i in range(0, 6):
    G.add_edge("usuario1", "twit" + str(i))
for i in range(7, 13):
    G.add_edge("usuario2", "twit" + str(i))
G.add_edge("usuario3", "twit13")

G.node_label = "Twits"
for i in range(13):
    G.add_edge("twit" + str(i), "hashtag1")
for i in range(1, 8):
    G.add_edge("twit13", "hashtag" + str(i))

# Draw graph

In [10]:
graph = StyledGraph(driver)

QUERY = "MATCH p=()-[]->() RETURN p"

graph.add_from_query(QUERY)
graph.draw()

In [11]:
LABEL_USUARIOS = "Usuarios"
LABEL_HASHTAGS = "Hashtag"
LABEL_TWITS = "Twits"
R_POSTS = "POSTS"
R_TAGS = "TAGS"

RED = "#ff0000"
ORANGE = "#f79767"
BLUE = "#57c7e3"
WHITE = "#FFFFFF"

for node in list(graph.nodes.values()):
    node["label"] = ""
    if LABEL_USUARIOS in node.label:
        node["value"] = len(node.edges)
        node["color"] = {
            "border": WHITE,
            "background": RED
        }
    elif LABEL_HASHTAGS in node.label:
        node["value"] = 1
        node["color"] = {
            "border": WHITE,
            "background": ORANGE
        }
    elif LABEL_TWITS in node.label:
        node["value"] = 1
        node["color"] = {
            "border": WHITE,
            "background": BLUE
        }
    else:
        # unknown label
        pass

for edge in graph.edges:
    edge["value"] = random.randrange(1, 10)
    edge["color"] = {
        "color": "#e8c2b0"
    }
    
node = list(graph.nodes.values())[1]
node["value"] = 8
node["shape"] = "triangle"
node["borderWidth"] = 10
node["label"] = "EXAMPLE"
node["font"] = {
    "color": "#e8c2b0",
    "size": 20,
}

In [12]:
node["color"]

{'border': '#FFFFFF', 'background': '#ff0000'}

In [13]:
graph.draw()

# Create graph

In [14]:
LABEL_USUARIOS = "Usuarios"
R_INTERACTS = "INTERACTS"

G.node_label = LABEL_USUARIOS
for i in range(3, 6):
    G.add_node("usuario" + str(i))
    
for i in range(1, 4):
    G.add_edge("usuario0", "usuario" + str(i))
G.add_edge("usuario3", "usuario4")
G.add_edge("usuario1", "usuario5")

In [15]:
G.add_edge("usuario1", "usuario5")

# draw graph

In [16]:
class Twits(StructuredRel):
    pass

In [17]:
class Posts(StructuredRel):
    pass

In [18]:
class Connected(StructuredRel):
    pass

In [19]:
class Usuarios(StructuredNode):
    id = StringProperty()
    # traverse outgoing IS_FROM relations, inflate to Country objects
    #posts = RelationshipTo(Twits, 'POSTS')
    #posts = RelationshipTo("Twits", "POSTS", model = Posts)
    connected = RelationshipTo("Usuarios", "CONNECTED", model = Connected)

In [20]:
QUERY = """ 
    MATCH (n1:Usuarios) MATCH (n2)-[r:CONNECTED]-(n3) WHERE id(n1) = id(n2) RETURN n1, count(r) as twitCount
"""

Z = neomodel.db.cypher_query(
    QUERY, 
    resolve_objects=True)

In [21]:
twitcount = defaultdict(lambda: 0)
for row in Z[0]:
    user_id = row[0].id
    num_twits = row[1]
    twitcount[user_id] = num_twits

In [22]:
QUERY = """ 
    MATCH (n1:Usuarios)-[r:CONNECTED]->() 
    RETURN r
"""

graph = StyledGraph(driver)
Z = neomodel.db.cypher_query(
    QUERY, 
    resolve_objects=True)
relationships_list = [rel[0] for rel in Z[0]]    
graph.add_from_neomodel_relationships(relationships_list)

### style graph

In [23]:
import string

def make_color_tuple( color ):
    """
    turn something like "#000000" into 0,0,0
    or "#FFFFFF into "255,255,255"
    """
    R = color[1:3]
    G = color[3:5]
    B = color[5:7]

    R = int(R, 16)
    G = int(G, 16)
    B = int(B, 16)

    return R,G,B

def interpolate_tuple( startcolor, goalcolor, steps ):
    """
    Take two RGB color sets and mix them over a specified number of steps.  Return the list
    """
    # white

    R = startcolor[0]
    G = startcolor[1]
    B = startcolor[2]

    targetR = goalcolor[0]
    targetG = goalcolor[1]
    targetB = goalcolor[2]

    DiffR = targetR - R
    DiffG = targetG - G
    DiffB = targetB - B

    buffer = []

    for i in range(0, steps +1):
        iR = R + int(DiffR * i / steps)
        iG = G + int(DiffG * i / steps)
        iB = B + int(DiffB * i / steps)

        hR = str(hex(iR)).replace("0x", "")
        hG = str(hex(iG)).replace("0x", "")
        hB = str(hex(iB)).replace("0x", "")

        if len(hR) == 1:
            hR = "0" + hR
        if len(hB) == 1:
            hB = "0" + hB

        if len(hG) == 1:
            hG = "0" + hG

        color = ("#"+hR+hG+hB).upper()
        buffer.append(color)

    return buffer

def interpolate( startcolor, goalcolor, steps ):
    """
    wrapper for interpolate_tuple that accepts colors as html ("#CCCCC" and such)
    """
    start_tuple = make_color_tuple(startcolor)
    goal_tuple = make_color_tuple(goalcolor)

    return interpolate_tuple(start_tuple, goal_tuple, steps)

In [24]:
RED = "#ff0000"
ORANGE = "#f79767"
BLUE = "#57c7e3"
WHITE = "#FFFFFF"

colors = interpolate(RED, BLUE, 10)

for node in graph.nodes.values():
    node["label"] = str(node.id)
    node["value"] = twitcount[node.id]
    node["color"] = {
        "border": WHITE,
        "background": random.choice(colors)
    }

In [25]:
graph.draw()

In [26]:
print(graph.options)

{'nodes': {'color': {'border': '#FFFFFF', 'background': '#F9A6C1'}, 'borderWidth': 2, 'borderWidthSelected': 2, 'shape': 'dot'}, 'edges': {'arrows': {'to': {'enabled': 'false', 'scaleFactor': 0.5}}, 'color': {'inherit': 'false'}, 'font': {'size': 14, 'align': 'middle'}, 'smooth': {'enabled': 'true', 'type': 'dynamic'}, 'length': 200}, 'interaction': {'dragNodes': 'true', 'hideEdgesOnDrag': 'false', 'hideNodesOnDrag': 'false'}, 'physics': {'enabled': 'true', 'stabilization': {'enabled': 'true', 'fit': 'true'}}}
